In [27]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from IPython.display import display, Markdown
from operator import itemgetter
import os

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, BaseChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.llms import Cohere
import gradio as gr

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableParallel
from langchain_community.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain_core.runnables import RunnableParallel
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

from getpass import getpass
from dotenv import load_dotenv

In [28]:
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
COHERE_API_KEY = os.environ["COHERE_API_KEY"]

In [5]:
loader = DirectoryLoader("data/sources_txt/")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30, separator=".")
docs = text_splitter.split_documents(documents=documents)

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

Created a chunk of size 455, which is longer than the specified 200
Created a chunk of size 212, which is longer than the specified 200
Created a chunk of size 260, which is longer than the specified 200
Created a chunk of size 240, which is longer than the specified 200
Created a chunk of size 221, which is longer than the specified 200
Created a chunk of size 243, which is longer than the specified 200
Created a chunk of size 260, which is longer than the specified 200
Created a chunk of size 215, which is longer than the specified 200
Created a chunk of size 242, which is longer than the specified 200
Created a chunk of size 240, which is longer than the specified 200
Created a chunk of size 221, which is longer than the specified 200
Created a chunk of size 243, which is longer than the specified 200
Created a chunk of size 218, which is longer than the specified 200
Created a chunk of size 407, which is longer than the specified 200
Created a chunk of size 203, which is longer tha

In [29]:
news = ['abc', 'elplural', 'elmundo', 'okdiario']

def get_clean_source(full_path):
    import re
    pattern = re.compile(r'\\([^\\]+)\.txt$')

    return pattern.search(full_path).group(1)

def create_context(query, news):
    from collections import defaultdict
    docs = []

    for n in ['abc', 'elplural', 'elmundo', 'okdiario']:
        docs = docs + (vectorstore.similarity_search_with_score(query, 4, filter=dict(source=f'data\\sources_txt\\{n}.txt')))

    grouped_texts = defaultdict(str)

    for doc in docs:
        source = doc[0].metadata.get('source')
        grouped_texts[source] += doc[0].page_content + ' '

    result_dict = dict(grouped_texts)
    rel_sources = [get_clean_source(n) for n in result_dict.keys()]


    resultado = "Las siguientes fuentes no contienen información al respecto: " + ", ".join([n for n in ['abc', 'elplural', 'elmundo', 'okdiario'] if n not in rel_sources])+ \
    """\n\nLas siguientes fuentes sí contienen información: 
    """ + '\n'.join([f"{get_clean_source(source)}: {content} \n\n" for source, content in result_dict.items()])

    return resultado

def consulta(q, model="gpt-3.5-turbo-0125", news=['abc', 'elplural', 'elmundo', 'okdiario']):
    from prompts.prompts import template_solojustificacion
    from prompts.prompts import template_clasificacion

    contexto = create_context(q, news)

    print(model)

    if model == 'cohere':
        llm=Cohere(model="command", temperature=0)

        response = llm.invoke(f"La afirmación es: {q}" + template_solojustificacion + contexto)
        # display(response['context'])
        # Markdown(response['answer'])
        response2 = llm.invoke(template_clasificacion + response)
        return response2, contexto

    else:
        llm = ChatOpenAI(model_name=model, # gpt-4-0125-preview gpt-3.5-turbo-0125
                    temperature=0,
                    openai_api_key=OPENAI_API_KEY)

        response = llm.invoke(f"La afirmación es: {q}" + template_solojustificacion + contexto)
        # display(response['context'])
        # Markdown(response['answer'])
        response2 = llm.invoke(template_clasificacion + response.content)
        return response2.content, contexto

In [ ]:
gr.Interface(consulta,
             inputs = [
                gr.Textbox(label="Afirmación", info="Inserte la información que desee verificar"),
                gr.Radio(["gpt-3.5-turbo-0125", "gpt-4-0125-preview", "cohere"], value="gpt-3.5-turbo-0125", label="Modelo", info="Los modelos más potentes suelen consumir más recursos"),
                ],
             outputs = [
                 gr.Markdown('textbox', label="Resultado"),
                 gr.Textbox(label="Referencias")
                 ],
             title='Verificando información desde la prensa',
             theme=gr.themes.Soft()).launch()